In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/featuresounddepression/htfatorch


In [2]:
import logging
import numpy as np
import htfa_torch.dtfa as DTFA
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
lepping_db = niidb.FMriActivationsDb('data/sound_depression.db', mask=None)

In [5]:
dtfa = DTFA.DeepTFA(lepping_db.all(_limit=2), mask=None, num_factors=100, embedding_dim=2)

01/11/2019 18:03:44 Loading Nifti image /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-mdd16_task-music_run-5_bold_space-MNI152NLin2009cAsym_preproc.nii.gz with mask None (zscore=True, smooth=None)


In [6]:
dtfa.normalize_activations()

[tensor(5.5638), tensor(5.5638)]

In [7]:
dtfa.visualize_factor_embedding()

(<nilearn.plotting.displays.OrthoProjector at 0x7f1f0d768390>,
 tensor([[-3.3275e+01, -8.4072e+01, -1.0107e+01],
         [ 4.8409e+01,  1.3786e+01,  9.8556e-01],
         [-1.1423e+00,  3.4979e+01,  1.1071e+01],
         [-1.6897e+01, -3.2753e+01,  4.0601e+01],
         [ 1.9064e+01, -5.3900e+01, -4.8700e+01],
         [ 3.6581e+01, -5.0746e+01,  5.3230e+01],
         [-2.3078e+01, -3.9232e+01, -3.1585e+01],
         [ 3.6350e+00,  3.0307e+01,  5.6705e+01],
         [-3.0916e+01, -6.4104e+00,  4.7960e+00],
         [ 2.4607e+01, -5.3547e+01,  1.4065e+01],
         [ 2.1705e+01, -6.0962e+01, -1.4893e+01],
         [ 4.5873e+01, -2.3632e+00,  4.8553e+01],
         [-3.8522e+01, -4.6417e+01,  5.7729e+01],
         [ 1.2747e+01, -6.8663e+01,  5.4958e+01],
         [-2.8661e+01, -7.8724e+00,  3.1794e+01],
         [-4.1380e+01,  4.0668e+01, -4.7891e+00],
         [ 2.5779e+01,  5.8524e+01,  1.5457e+01],
         [ 3.5996e+01, -8.0206e+01,  1.4677e+01],
         [ 4.0165e+01,  3.7411e+01,  

In [8]:
losses = dtfa.train(num_steps=2, learning_rate=1e-2, log_level=logging.INFO, num_particles=1,
                    batch_size=25, use_cuda=True, checkpoint_steps=50, blocks_batch_size=4, patience=100)

01/11/2019 18:06:11 [Epoch 1] (120ms) Posterior free-energy 2.48175725e+06 = KL from prior 1.46418839e+01 - log-likelihood -2.48174250e+06
01/11/2019 18:06:11 Saving checkpoint...
01/11/2019 18:06:11 [Epoch 2] (106ms) Posterior free-energy 2.45470975e+06 = KL from prior 3.02325478e+01 - log-likelihood -2.45467950e+06
01/11/2019 18:06:11 Saving checkpoint...


In [9]:
utils.plot_losses(losses)

In [10]:
for b in np.random.choice(range(dtfa.num_blocks), 5):
    dtfa.plot_original_brain(b)
    dtfa.plot_reconstruction(b)

01/11/2019 18:06:21 Reconstruction Error (Frobenius Norm): 1.15069202e+03 out of 1.01454266e+03
01/11/2019 18:06:25 Reconstruction Error (Frobenius Norm): 1.12537415e+03 out of 1.01454266e+03
01/11/2019 18:06:27 Reconstruction Error (Frobenius Norm): 3.50006165e+02 out of 3.14628265e+02
01/11/2019 18:06:30 Reconstruction Error (Frobenius Norm): 3.64608490e+02 out of 3.14628265e+02
01/11/2019 18:06:34 Reconstruction Error (Frobenius Norm): 1.14568506e+03 out of 1.01454266e+03


In [ ]:
def task_labeler(b):
    return b.task

In [ ]:
dtfa.scatter_task_embedding(labeler=task_labeler, figsize=None)

In [ ]:
def subject_labeler(block):
    return 'Subject %d' % block.subject

def subject_run_labeler(block):
    return 'sub%drun%d' % (block.subject, block.run)

In [ ]:
dtfa.scatter_subject_embedding(labeler=subject_labeler, figsize=None)

In [ ]:
dtfa.visualize_factor_embedding()

In [ ]:
dtfa.average_reconstruction_error()